<a href="https://colab.research.google.com/github/baban9/MapReduce-/blob/master/multithreading_of_ids_561_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/Shared drives/IDS 561 - Big Data /Assignment 1 ')

!ls

'Assignment 1.ipynb'   'ids 561 Assignment 1.ipynb'   pride.txt
'Frequency count.csv'   IDS561_lab2.ipynb	     'Word count.csv'


In [ ]:
import string
import re
from operator import itemgetter
from collections import defaultdict
import pandas as pd
import concurrent.futures
import threading


In [ ]:
def data_cleaning(file_name):
  # Some data cleaning jobs, such as  removing numbers, punctuations and special symbols, uppercase to lower case
  line = open(file_name, 'rt')
  text = line.read().strip()
  
  # removing numbers, punctuations, and special symbols
  text_processing = re.sub('[^a-zA-Z\.]', ' ', text)
  text_only = re.sub("  "," ", text_processing)

  # lower case characters 
  text_processed = text_only.lower()

  return text_processed

def data_split(processed_text):
  # Split the dataset into two parts: Part1 includes the first 5000 lines of the text file, Part2 includes the rest text.
  lines =  text_processed.split(".")

  part1_data = lines[0:5000] 
  text_a = ' '.join([str(e) for e in part1_data]) 

  part2_data = lines[5001:]
  text_b = ' '.join([str(e) for e in part2_data])

  return text_a, text_b

def Mapper(text):
    mapped = []
    for word in text.split(" "):
      if word !="":
        mapped.append((word,1))
    
    return mapped


def Sorting(mapped_1, mapped_2):
  combined = mapped_1 + mapped_2
  sorted_words = sorted(combined, key=lambda x: x[0])

  return sorted_words

def partition(words_sorted):
  # partitioning 

  index = [x for x, y in enumerate(words_sorted) if y[0].startswith("n")][0]
  partition_a_m = words_sorted[0:index]
  partition_n_z = words_sorted[index+1:]

  return partition_a_m, partition_n_z


def Reducer(sorted_list):
    grouped = defaultdict(int) 

    for word in sorted_list:
      grouped[word[0]] +=1

    return grouped


def output(word_dict_a_m, word_dict_n_z):
  output = {**word_dict_a_m, **word_dict_n_z}
  df = pd.DataFrame(output.items(),columns = ['word','frequency'])

  return df

if __name__ == "__main__":
  processed_text = data_cleaning('pride.txt')
  text_5000, text_rest = data_split( processed_text)

  # mapper threads
  with concurrent.futures.ThreadPoolExecutor() as executor:
    mapped_a = executor.submit(Mapper, text_5000)
    mapped_b = executor.submit(Mapper, text_rest)

  mapped_5000 = mapped_a.result()
  mapped_rest = mapped_b.result()

  words_sorted = Sorting(mapped_5000, mapped_rest)

  partition_a_m, partition_n_z = partition(words_sorted)

  # reducer threads
  with concurrent.futures.ThreadPoolExecutor() as executor:
    word_dict_thread_a= executor.submit(Reducer, partition_a_m)
    word_dict_thread_n= executor.submit(Reducer, partition_n_z)

  word_dict_a_m = word_dict_thread_a.result()
  word_dict_n_z = word_dict_thread_n.result()

  data = output(word_dict_a_m,word_dict_n_z)

  df_sorted_word = data.copy() 
  df_sorted_freq = data.sort_values(['frequency'], ascending=False)

  df_sorted_word.to_csv("/content/drive/Shared drives/IDS 561 - Big Data /Assignment 1 /Word count.csv")
  df_sorted_freq.to_csv("/content/drive/Shared drives/IDS 561 - Big Data /Assignment 1 /Frequency count.csv")

  


In [ ]:
# approach 2 


# Some data cleaning jobs, such as  removing numbers, punctuations and special symbols, uppercase to lower case
line = open("pride.txt", 'rt')
text = line.read().strip()

# removing numbers, punctuations, and special symbols
text_processing = re.sub('[^a-zA-Z\.]', ' ', text)
text_only = re.sub("  "," ", text_processing)

# lower case characters 
text_processed = text_only.lower()

# Split the dataset into two parts: Part1 includes the first 5000 lines of the text file, Part2 includes the rest text.
lines =  text_processed.split(".")

part1_data = lines[0:5000] 
text_a = ' '.join([str(e) for e in part1_data]) 

part2_data = lines[5001:]
text_b = ' '.join([str(e) for e in part2_data])

with concurrent.futures.ThreadPoolExecutor() as executor:
  mapped_a = executor.submit(Mapper, text_a)
  mapped_b = executor.submit(Mapper, text_b)

mapped_5000 = mapped_a.result()
mapped_rest = mapped_b.result()

    
# Sorting function 
combined = mapped_5000 + mapped_rest
sorted_words = sorted(combined, key=lambda x: x[0])

# partitioning 

index = [x for x, y in enumerate(sorted_words) if y[0].startswith("n")][0]
partition_a_m = sorted_words[0:index]
partition_n_z = sorted_words[index+1:]

# reducer 
with concurrent.futures.ThreadPoolExecutor() as executor:
  word_dict_thread_a= executor.submit(Reducer, partition_a_m)
  word_dict_thread_n= executor.submit(Reducer, partition_n_z)

word_dict_a_m = word_dict_thread_a.result()
word_dict_n_z = word_dict_thread_n.result()
    
output = {**word_dict_a_m, **word_dict_n_z}

df = pd.DataFrame(output.items(),columns = ['word','frequency'])

df_sorted_word = df.copy() 
df_sorted_freq = df.sort_values(['frequency'], ascending=False)

df_sorted_word.to_csv("/content/drive/Shared drives/IDS 561 - Big Data /Assignment 1 /Word count.csv")
df_sorted_freq.to_csv("/content/drive/Shared drives/IDS 561 - Big Data /Assignment 1 /Frequency count.csv")
